In [3]:
import pandas as pd 

data = pd.read_csv('./gladys_after_extracting_findings.csv')

In [4]:
from thang_utils import *  

In [5]:
data.head()

,Filename,Output,Filename(2nd edition,Output.1,REFLACX,EyeGaze,Findings From The Ground Truth,Questions About The Findings,Response,Questions About the Location of the Findings,Response for the Locations of the findings
0,fa771fa1-d9571d07-bff8f655-327734a7-6e10b29d,Chest X-ray findings\nView: Portable AP chest ...,fa771fa1-d9571d07-bff8f655-327734a7-6e10b29d,This chest X-ray shows increased opacity in th...,left picc line tip is in the left brachiocepha...,NaN,1. PICC line tip \n\n2. Bilateral Pleural effu...,1. Do you see PICC line tip in the above chest...,"1. In the chest X-ray provided, a peripherally...",1. Where do you see PICC line tip in the above...,1. The PICC line tip appears to be located in ...
1,590ae2db-d9330aec-1e81d571-d6bd3751-640cee9b,Chest X-ray findings\nLungs & pleura:\n\nThere...,590ae2db-d9330aec-1e81d571-d6bd3751-640cee9b,This is a lateral chest X-ray. There is hyperi...,NaN,""" sternotomy wires with cardiac clips. post su...",1. sternotomy wires with cardiac clips ...,1. Do you see sternotomy wires with cardiac cl...,"1. Yes, the chest X-ray image shows the prese...",1. Where do you see sternotomy wires with card...,1. The sternotomy wires are visible in the m...
2,4873aa08-977bfd31-fb492e64-6ef432d1-3f12cbe3,Chest X-ray findings\nView: Frontal (PA or AP)...,4873aa08-977bfd31-fb492e64-6ef432d1-3f12cbe3,This chest X-ray shows evidence of prior stern...,NaN,""" sternotomy wires with cardiac clips. post su...",1. sternotomy wires with cardiac clips ...,1. Do you see sternotomy wires with cardiac cl...,"1. Yes, the chest X-ray image shows the prese...",1. Where do you see sternotomy wires with card...,"1. In the provided chest X-ray, multiple ster..."
3,dcdc4bd9-4301b111-2a65a814-ee8e7bc5-7f0b9a5a,Chest X-ray findings\nView: Portable AP chest ...,dcdc4bd9-4301b111-2a65a814-ee8e7bc5-7f0b9a5a,This chest X-ray shows bilateral medical devic...,support apparatus. no pneumothorax. enlarged c...,NaN,\n1. Support apparatus present \n2. ...,1. Do you see a support apparatus present in t...,"1. Yes, the chest X-ray image shows the presen...",1. Where do you see a support apparatus presen...,1. There is a central venous catheter presen...
4,5bdabba9-388f6646-ac06b5f5-f68b2fd2-3630de21,Chest X-ray findings\nView: Upright AP chest r...,5bdabba9-388f6646-ac06b5f5-f68b2fd2-3630de21,This chest X-ray shows a frontal (AP) view tak...,no acute parenchyma or pleural abnormalities. ...,NaN,1. No acute parenchyma \n2. ...,1. Do you see an acute parenchyma in the above...,1. This chest X-ray shows diffuse bilateral o...,1. Where do you see an acute parenchyma in the...,1. The acute parenchymal opacity appears to...


In [23]:
def normalize_findings_string(finding_str):
    if pd.isna(finding_str):
        return ""
    
    # Split by lines and clean each line
    lines = finding_str.split('\n')
    normalized_lines = []
    
    for line in lines:
        # Strip whitespace and extra spaces
        cleaned_line = ' '.join(line.strip().split())
        if cleaned_line:  # Only add non-empty lines
            normalized_lines.append(cleaned_line)
    
    return '. '.join(normalized_lines)

def split_findings_into_items(finding_str):
    if pd.isna(finding_str) or not finding_str.strip():
        return []
    
    # Normalize the string first
    normalized = normalize_findings_string(finding_str)
    
    # Split by common numbering patterns
    
    # Pattern to match: number/letter followed by period and optional space
    pattern = r'(?:^|\s)(?:\d+\.|[a-zA-Z]\.)(?:\s*)'
    
    # Split by the pattern
    items = re.split(pattern, normalized)
    
    # Clean up the items - remove empty strings and strip whitespace
    cleaned_items = []
    for item in items:
        cleaned = item.strip()
        if cleaned and cleaned != '.':  # Remove empty items and standalone periods
            # Remove trailing punctuation
            cleaned = re.sub(r'[.,:;]+$', '', cleaned)
            cleaned_items.append(cleaned)
    
    return cleaned_items
dic_dicom_to_finding = get_infinite_dict()
for index, row in data.iterrows():
    filename = row['Filename(2nd edition ']
    finding = row['Findings From The Ground Truth']
    if pd.notna(finding):  # Only add if finding is not NaN
        dic_dicom_to_finding[filename] = split_findings_into_items(finding)

In [24]:
save_as_json(dic_dicom_to_finding, './dic_dicom_to_finding.json')

In [25]:
dic_dicom_to_finding = open_as_json('./dic_dicom_to_finding.json')

In [26]:
make_question_exist = lambda x: f"Do you see {x} in the given CXR?"
make_question_where = lambda x: f"Where do you see {x} in the image? Only answer the region location from this list: [left lung, right lung, cardiac silhouette, mediastinum, left lower lung zone, right lower lung zone, right hilar structures, left hilar structures, upper mediastinum, left costophrenic angle, right costophrenic angle, left mid lung zone, right mid lung zone, aortic arch, right upper lung zone, left upper lung zone, right hemidiaphragm, right clavicle, left clavicle, left hemidiaphragm, right apical zone, trachea, left apical zone, carina, svc, right atrium, cavoatrial junction, abdomen, spine]." 

In [27]:
all_jps = open_as_json('jpg_files_list.json')

In [28]:
dic_dicom_to_question = get_infinite_dict()
# Create mapping from dicom filename to full path
dicom_to_path = {}
for path in all_jps['file_paths']:
    for dicom_filename in dic_dicom_to_finding.keys():
        if dicom_filename in path:
            dicom_to_path[dicom_filename] = path
            break

for filename, findings_list in dic_dicom_to_finding.items():
    exist_questions = []
    location_questions = []
    
    for finding in findings_list:
        exist_questions.append(make_question_exist(finding))
        location_questions.append(make_question_where(finding))
    
    dic_dicom_to_question[filename] = {
        'exist_questions': exist_questions,
        'location_questions': location_questions,
        'path': dicom_to_path[filename]
    }

In [29]:
dic_dicom_to_question

defaultdict(<function thang_utils.utils.get_infinite_dict.<locals>.<lambda>()>,
            {'fa771fa1-d9571d07-bff8f655-327734a7-6e10b29d': {'exist_questions': ['Do you see PICC line tip in the given CXR?',
               'Do you see Bilateral Pleural effusions in the given CXR?',
               'Do you see Suspected Atelectasis in the given CXR?',
               'Do you see Suspected Consolidation in the given CXR?',
               'Do you see No pneumothorax in the given CXR?',
               'Do you see Normal heart size in the given CXR?',
               'Do you see Normal pulmonary vessels in the given CXR?',
               'Do you see Normal osseous structures in the given CXR?'],
              'location_questions': ['Where do you see PICC line tip in the image? Only answer the region location from this list: [left lung, right lung, cardiac silhouette, mediastinum, left lower lung zone, right lower lung zone, right hilar structures, left hilar structures, upper mediastinum, left

In [30]:
save_as_json(dic_dicom_to_question,'./dic_dicom_to_question.json')

In [32]:
# Create pairs of (dicom, question) for all questions
dicom_question_pairs = []

for dicom_filename, question_data in dic_dicom_to_question.items():
    # Add all exist questions
    for question in question_data['exist_questions']:
        dicom_question_pairs.append((question_data['path'], question))
    
    # Add all location questions
    for question in question_data['location_questions']:
        dicom_question_pairs.append((question_data['path'], question))

save_as_json(dicom_question_pairs, './dicom_question_pairs.json')